<a href="https://colab.research.google.com/github/jbcgames/Lab_1_Controls/blob/main/ResumenM%C3%A9todo_de_Smith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Informe de Identificación de Modelo FOPDT utilizando el Método de Smith

##Introducción

Este informe detalla el proceso de identificación de un modelo FOPDT (First Order Plus Dead Time) para un sistema térmico utilizando el método de Smith. El análisis se realizó a partir de datos experimentales obtenidos de un sistema de control de temperatura con sensores LM35 y calentadores controlados por PWM.

####Importación de Bibliotecas

In [ ]:
! pip install pandas numpy matplotlib control

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

##Lectura de Datos

In [ ]:
url_datos = 'https://raw.githubusercontent.com/jbcgames/Lab_1_Controls/main/Datos.txt'
df = pd.read_csv(url_datos)

Propósito: Cargar los datos experimentales desde un repositorio en línea.

Detalles: Los datos se almacenan en un DataFrame de pandas para su posterior procesamiento.

##Extracción y Filtrado de Datos

In [ ]:
tiempo = df['Tiempo (s)'].values
T1 = df[' T1 (C)'].values
pwm = df[' PWM_Heater1'].values
mask = (tiempo <= 450)
tiempo = tiempo[mask]
T1 = T1[mask]
pwm = pwm[mask]

Propósito: Extraer las columnas relevantes (tiempo, temperatura T1 y señal PWM) y limitar el análisis a los primeros 450 segundos.

Detalles: Se crea una máscara booleana para seleccionar solo los datos dentro del rango de tiempo deseado.

##Detección del Escalón PWM

In [ ]:
indices_escalon = np.where((pwm[:-1] == 0) & (pwm[1:] > 0))[0]
if len(indices_escalon) == 0:
    print("No se detectó un escalón (0 -> valor>0) en PWM.")
    exit()
idx_escalon = indices_escalon[0]
t_escalon = tiempo[idx_escalon + 1]
print(f"Escalón detectado en t = {t_escalon:.2f} s")

Propósito: Identificar el momento exacto en que se aplica un escalón de potencia al sistema (transición de PWM=0 a PWM>0).

Detalles: Se buscan los índices donde ocurre esta transición y se registra el tiempo correspondiente.

##Recorte de Datos

In [ ]:
tiempo = tiempo[idx_escalon:]
T1 = T1[idx_escalon:]
pwm = pwm[idx_escalon:]

Propósito: Ajustar los datos para comenzar el análisis desde el instante del escalón.

Detalles: Se descartan los datos anteriores al escalón para centrarse en la respuesta del sistema.

##Filtrado de Señal

In [ ]:
window_length = 31
polyorder = 3
T1_smooth = savgol_filter(T1, window_length, polyorder)

Propósito: Suavizar los datos de temperatura para reducir el ruido y facilitar la identificación del modelo.

Detalles: Se aplica un filtro Savitzky-Golay con ventana de 31 puntos y polinomio de orden 3.

##Determinación de Valores Iniciales y Finales

In [ ]:
y0 = T1_smooth[0]
yf = T1_smooth[-1]
delta_y = yf - y0

Propósito: Calcular el cambio total en temperatura causado por el escalón.

Detalles: Se toman los valores inicial (y0) y final (yf) de la señal suavizada para determinar el rango de variación.

##Cálculo de Puntos Característicos

In [ ]:
y_28 = y0 + 0.283 * delta_y
y_63 = y0 + 0.632 * delta_y
t_28 = tiempo[np.argmin(np.abs(T1_smooth - y_28))]
t_63 = tiempo[np.argmin(np.abs(T1_smooth - y_63))]

Propósito: Encontrar los tiempos en que la respuesta alcanza el 28.3% y 63.2% de su cambio total.

Detalles: Estos puntos son cruciales para el método de Smith, ya que permiten estimar los parámetros del modelo.

##Cálculo de Parámetros del Modelo

In [ ]:
tau = 1.5 * (t_63 - t_28)
L = t_63 - tau
K = delta_y / pwm[idx_escalon + 1]

Propósito: Calcular la constante de tiempo (τ), el tiempo muerto (L) y la ganancia (K) del modelo FOPDT.

Detalles:

-τ se calcula como 1.5 veces la diferencia entre t_63 y t_28.

-L se obtiene restando τ de t_63.

-K es la relación entre el cambio en temperatura y el cambio en PWM.

##Simulación del Modelo

In [ ]:
t_modelo = tiempo
respuesta_modelo = np.zeros_like(t_modelo)
for i, t in enumerate(t_modelo):
    if t < L:
        respuesta_modelo[i] = y0
    else:
        respuesta_modelo[i] = y0 + K * pwm[idx_escalon + 1] * (1 - np.exp(-(t - L)/tau))

Propósito: Generar la respuesta teórica del modelo FOPDT identificado.

Detalles: Se implementa la ecuación del modelo FOPDT considerando el tiempo muerto L.

##Visualización de Resultados

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(tiempo, T1, label='Datos originales', alpha=0.7)
plt.plot(tiempo, T1_smooth, label='Datos suavizados (Savitzky-Golay)')
plt.plot(t_modelo, respuesta_modelo, '--', label='Modelo FOPDT (Smith)')
plt.axvline(L, color='gray', linestyle='--', label='Tiempo muerto L')

Propósito: Comparar visualmente los datos experimentales con el modelo identificado.

Detalles: Se grafican los datos originales, los suavizados y la respuesta del modelo, junto con una línea que indica el tiempo muerto.

##Resultados Obtenidos

-Los datos originales de temperatura con cierto ruido.

-La señal suavizada que facilita la identificación de parámetros.

-La respuesta del modelo FOPDT identificado.

-La línea vertical que indica el tiempo muerto estimado.

##Conclusiones

El modelo FOPDT obtenido mediante el método de Smith (K=0.361, τ=135.26 s, L=25.07 s) representa satisfactoriamente el comportamiento del sistema térmico. Los parámetros identificados muestran una relación estable entre la potencia aplicada y la temperatura, con la respuesta lenta característica de estos sistemas y un retardo asociado a la inercia térmica. La concordancia entre los datos experimentales procesados y la respuesta del modelo confirma la validez del enfoque, aunque se observan pequeñas diferencias que podrían atribuirse a efectos secundarios no considerados. Estos resultados constituyen un punto de partida sólido para el desarrollo de estrategias de control, recomendándose complementar el estudio con pruebas adicionales para verificar el modelo en diferentes condiciones de operación.